In [1]:
!pip3 install pyathena

In [2]:
from pyathena import connect
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.linear_model import LinearRegression

#metricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


# Carregando os Dados

In [3]:
conn = connect(s3_staging_dir='s3://techchallenge-fiap/output/',
               region_name='us-east-1')

In [4]:
df = pd.read_sql('SELECT * FROM "default"."techchallenge3";', conn)

/tmp/ipykernel_9181/3493947035.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM "default"."techchallenge3";', conn)


# Explorando os Dados

In [5]:
df

,id,model,year,mileage,condition,price,make
0,1,Silverado,2014,13578.0,Excellent,27321.10,Toyota
1,6,Altima,2014,11470.0,Excellent,27426.60,Toyota
2,10,Altima,2019,112693.0,Excellent,17365.45,Toyota
3,12,Silverado,2012,79902.0,Excellent,26005.00,Toyota
4,14,F-150,2014,72976.0,Excellent,24351.30,Toyota
...,...,...,...,...,...,...,...
995,971,F-150,2013,98891.0,Excellent,24055.45,Nissan
996,985,Altima,2022,41167.0,Excellent,17941.75,Nissan
997,987,Altima,2013,109094.0,Excellent,23545.40,Nissan
998,995,Camry,2010,149032.0,Excellent,24548.50,Nissan


In [6]:
df.head()

,id,model,year,mileage,condition,price,make
0,1,Silverado,2014,13578.0,Excellent,27321.10,Toyota
1,6,Altima,2014,11470.0,Excellent,27426.60,Toyota
2,10,Altima,2019,112693.0,Excellent,17365.45,Toyota
3,12,Silverado,2012,79902.0,Excellent,26005.00,Toyota
4,14,F-150,2014,72976.0,Excellent,24351.30,Toyota


In [7]:
df.shape[0]

1000

In [8]:
df.shape[1]

7

In [9]:
df.isnull().sum()

id           0
model        0
year         0
mileage      0
condition    0
price        0
make         0
dtype: int64

In [10]:
df.describe()


,id,year,mileage,price
count,1000.000000,1000.00000,1000.000000,1000.000000
mean,499.500000,2015.86500,78796.927000,22195.205650
std,288.819436,3.78247,39842.259941,4245.191585
min,0.000000,2010.00000,10079.000000,12613.000000
25%,249.750000,2013.00000,44942.750000,18961.862500
50%,499.500000,2016.00000,78056.500000,22247.875000
75%,749.250000,2019.00000,112366.250000,25510.275000
max,999.000000,2022.00000,149794.000000,31414.900000


# Separação dos Dados de Treino e Teste

In [11]:
df = df.dropna()

Convertendo variáveis categóricas em numerais.

In [12]:
df = pd.get_dummies(df)

In [13]:
df

,id,year,mileage,price,model_Altima,model_Camry,model_Civic,model_F-150,model_Silverado,condition_Excellent,condition_Fair,condition_Good,make_Chevrolet,make_Ford,make_Honda,make_Nissan,make_Toyota
0,1,2014,13578.0,27321.10,0,0,0,0,1,1,0,0,0,0,0,0,1
1,6,2014,11470.0,27426.60,1,0,0,0,0,1,0,0,0,0,0,0,1
2,10,2019,112693.0,17365.45,1,0,0,0,0,1,0,0,0,0,0,0,1
3,12,2012,79902.0,26005.00,0,0,0,0,1,1,0,0,0,0,0,0,1
4,14,2014,72976.0,24351.30,0,0,0,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,971,2013,98891.0,24055.45,0,0,0,1,0,1,0,0,0,0,0,1,0
996,985,2022,41167.0,17941.75,1,0,0,0,0,1,0,0,0,0,0,1,0
997,987,2013,109094.0,23545.40,1,0,0,0,0,1,0,0,0,0,0,1,0
998,995,2010,149032.0,24548.50,0,1,0,0,0,1,0,0,0,0,0,1,0


In [14]:
X = df.drop('price', axis=1)

In [15]:
X

,id,year,mileage,model_Altima,model_Camry,model_Civic,model_F-150,model_Silverado,condition_Excellent,condition_Fair,condition_Good,make_Chevrolet,make_Ford,make_Honda,make_Nissan,make_Toyota
0,1,2014,13578.0,0,0,0,0,1,1,0,0,0,0,0,0,1
1,6,2014,11470.0,1,0,0,0,0,1,0,0,0,0,0,0,1
2,10,2019,112693.0,1,0,0,0,0,1,0,0,0,0,0,0,1
3,12,2012,79902.0,0,0,0,0,1,1,0,0,0,0,0,0,1
4,14,2014,72976.0,0,0,0,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,971,2013,98891.0,0,0,0,1,0,1,0,0,0,0,0,1,0
996,985,2022,41167.0,1,0,0,0,0,1,0,0,0,0,0,1,0
997,987,2013,109094.0,1,0,0,0,0,1,0,0,0,0,0,1,0
998,995,2010,149032.0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [16]:
y = df['price']

In [17]:
y

0      27321.10
1      27426.60
2      17365.45
3      26005.00
4      24351.30
         ...   
995    24055.45
996    17941.75
997    23545.40
998    24548.50
999    18580.60
Name: price, Length: 1000, dtype: float64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=25)

In [19]:
X_train

,id,year,mileage,model_Altima,model_Camry,model_Civic,model_F-150,model_Silverado,condition_Excellent,condition_Fair,condition_Good,make_Chevrolet,make_Ford,make_Honda,make_Nissan,make_Toyota
357,731,2016,111433.0,0,0,0,0,1,0,0,1,1,0,0,0,0
114,552,2015,49624.0,0,0,0,1,0,1,0,0,0,0,0,0,1
268,348,2016,13492.0,0,1,0,0,0,1,0,0,1,0,0,0,0
665,311,2014,89643.0,0,0,0,0,1,1,0,0,0,0,1,0,0
856,209,2019,63399.0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,559,2017,149439.0,0,0,0,0,1,0,0,1,1,0,0,0,0
143,716,2013,102901.0,1,0,0,0,0,1,0,0,0,0,0,0,1
474,271,2020,68319.0,0,0,1,0,0,1,0,0,0,1,0,0,0
318,560,2018,112050.0,0,1,0,0,0,1,0,0,1,0,0,0,0


In [20]:
X_test

,id,year,mileage,model_Altima,model_Camry,model_Civic,model_F-150,model_Silverado,condition_Excellent,condition_Fair,condition_Good,make_Chevrolet,make_Ford,make_Honda,make_Nissan,make_Toyota
688,421,2020,11052.0,0,0,0,0,1,1,0,0,0,0,1,0,0
49,217,2014,148505.0,0,1,0,0,0,0,0,1,0,0,0,0,1
288,437,2020,133725.0,0,0,0,1,0,0,0,1,1,0,0,0,0
698,463,2019,63167.0,0,0,0,0,1,1,0,0,0,0,1,0,0
775,810,2013,36782.0,0,1,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,161,2013,131297.0,0,0,0,0,1,1,0,0,0,0,1,0,0
209,16,2017,48281.0,0,0,0,1,0,0,1,0,1,0,0,0,0
296,469,2020,64646.0,1,0,0,0,0,0,1,0,1,0,0,0,0
609,976,2013,66767.0,1,0,0,0,0,1,0,0,0,1,0,0,0


In [21]:
y_train

357    20428.35
114    24518.90
268    25325.50
665    23517.95
856    19830.15
         ...   
317    17528.05
143    23854.85
474    18584.15
318    18397.60
132    29649.40
Name: price, Length: 700, dtype: float64

In [22]:
y_test

688    21447.50
49     20574.75
288    15313.85
698    19841.65
775    27160.90
         ...   
635    22435.15
209    22586.05
296    18767.80
609    25661.65
63     20390.00
Name: price, Length: 300, dtype: float64

# Regressão Linear

In [23]:
modelo = LinearRegression()

In [24]:
modelo.fit(X_train,y_train)

LinearRegression()

In [25]:
y_pred_lr = modelo.predict(X_test)

In [26]:
y_pred_lr

array([21447.4612199 , 20574.79656769, 15313.79591937, 19841.70849804,
       27160.95035226, 29352.99859003, 19469.51393269, 22499.35157378,
       19113.00172897, 31132.75954936, 21023.51562379, 20819.40109328,
       22233.73723521, 16771.20944552, 16575.86110926, 27519.15295778,
       25055.70970697, 30136.81113966, 19616.94743684, 27112.04576582,
       23187.70890272, 21606.46377356, 22142.31070753, 29208.75874976,
       18560.74484819, 24699.29637098, 16643.61824519, 25810.64007055,
       17467.10094592, 16620.24885569, 24743.64914655, 19470.29667817,
       18765.54505766, 21553.5095736 , 25360.85649843, 20855.30226892,
       23028.71400542, 20172.20453043, 18376.87431196, 19804.89305192,
       17225.95787859, 20904.34094388, 25769.10811119, 31274.76979151,
       25322.05985883, 23856.05066383, 21851.55042422, 20900.0702692 ,
       20665.25505574, 19796.5611261 , 27583.8099588 , 17086.86819109,
       28286.77108762, 25668.99807465, 27963.4177967 , 17165.21243023,
      

In [27]:
y_test

688    21447.50
49     20574.75
288    15313.85
698    19841.65
775    27160.90
         ...   
635    22435.15
209    22586.05
296    18767.80
609    25661.65
63     20390.00
Name: price, Length: 300, dtype: float64

# Calculando Desvios/Erros

In [28]:
mse = mean_squared_error(y_test,y_pred_lr, squared=True)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test,y_pred_lr)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
print(f'MSE: {mse}\nRMSE: {rmse}\nMAPE: {mape}')

MSE: 0.004873012460303519
RMSE: 0.06980696570044796
MAPE: 2.8416008945182665e-06
